In [1]:
import torch
import torch.nn as nn


In [2]:
inputs=torch.Tensor(1,1,28,28)
print('텐서의 크기 {}'.format(inputs.shape))

텐서의 크기 torch.Size([1, 1, 28, 28])


In [3]:
conv1=nn.Conv2d(1,32,3,padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
conv2=nn.Conv2d(32,64,kernel_size=3,padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
pool=nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [6]:
out=conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [7]:
out=pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
out=conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [9]:
out=pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [10]:
out.size(0)

1

In [11]:
out=out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [12]:
fc=nn.Linear(3136,10)
out=fc(out)
print(out.shape)

torch.Size([1, 10])


In [13]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [14]:
device ='cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device=='cuda':
  torch.cuda.manual_seed_all(777)

In [15]:
learning_rate=0.001
training_epochs=15
batch_size=100


In [18]:
mnist_train=dsets.MNIST(root='MNIST_data/',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)
mnist_test=dsets.MNIST(root='MNIST_data/',
                       train=False,
                       transform=transforms.ToTensor(),
                       download=True)

In [19]:
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True)

In [20]:
class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN,self).__init__()


    self.layer1=torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2))
    
    self.layer2=torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2))
    
    self.fc=torch.nn.Linear(7*7*64,10,bias=True)

    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=out.view(out.size(0),-1)
    out=self.fc(out)
    return out

In [21]:
model=CNN().to(device)

In [22]:
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)


In [23]:
total_batch=len(data_loader)

In [24]:
print('총 배치의 수 {}'.format(total_batch))

총 배치의 수 600


In [26]:
for epoch in range(training_epochs):
  avg_cost=0


  for X, Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)

    optimizer.zero_grad()
    hypothesis=model(X)
    cost=criterion(hypothesis,Y)
    cost.backward()
    optimizer.step()

    avg_cost+=cost/total_batch
  print('[Epoch: {:>4}] cost ={:>9}'.format(epoch+1,avg_cost))

[Epoch:    1] cost =0.22018921375274658
[Epoch:    2] cost =0.059701960533857346
[Epoch:    3] cost =0.04463321343064308
[Epoch:    4] cost =0.036017339676618576
[Epoch:    5] cost =0.030005617067217827
[Epoch:    6] cost =0.024593809619545937
[Epoch:    7] cost =0.021002544090151787
[Epoch:    8] cost =0.017881987616419792
[Epoch:    9] cost =0.014783747494220734
[Epoch:   10] cost =0.011866346932947636
[Epoch:   11] cost =0.0113667668774724
[Epoch:   12] cost =0.008658207021653652
[Epoch:   13] cost =0.007839069701731205
[Epoch:   14] cost =0.00767067214474082
[Epoch:   15] cost =0.006145003251731396


In [31]:
with torch.no_grad():
  X_test=mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
  Y_test=mnist_test.test_labels.to(device)

  prediction=model(X_test)
  correct_prediction=torch.argmax(prediction,1)==Y_test
  accuracy=correct_prediction.float().mean()
  print('Accuarcy',accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuarcy 0.9842000007629395


#깊은 CNN으로 MNIST 분류하기



# 모델 이해하기
합성곱 (in_channel=1,out_channel=32,kernel_size-3,stride=1,padding=1)+활성화 함수 ReLU 맥스풀링(kernel_size=2,stride=2))

합성곱 (in_channel=32,out_channel=64,kenner_size=3,stride=1,padding=1)+활성화 함수 ReLU 맥스풀링(kernel_size=2,stride=2))

합성곱 (in_channel=64, out_channel=128, kernel_size=3,stride=1,padding=1)+활성화 함수 ReLU 맥스풀링(kernel_size=2,stride=2,padding=1))

전결합층 , 
특성맵을 펼친다. #batch_size*4*4*128 -> batch_size*2048
전결합층(뉴런 625개) + 활성화 함수 ReLU

전결합층 (뉴런 10개) + 활성화 함수 Softmax 함수


In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init



In [2]:
device='cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device=='cuda':
  torch.cuda.manual_seed_all(777)

In [3]:
learning_rate=0.001
training_epochs=15
batch_size=100



In [4]:
mnist_train=dsets.MNIST(root='MNIST_data/',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)

mnist_test=dsets.MNIST(root='MNIST_data/',
                       train=False,
                       transform=transforms.ToTensor(),
                       download=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN,self).__init__()
    self.keep_prob=0.5
    
    self.layer1=torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2))
    
    self.layer2=torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2))
    
    self.layer3=torch.nn.Sequential(
        torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2,padding=1))
    

    self.fc1=torch.nn.Linear(4*4*128,625,bias=True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4=torch.nn.Sequential(
        self.fc1,
        torch.nn.ReLU(),
        torch.nn.Dropout(p=1-self.keep_prob))
    
    self.fc2=torch.nn.Linear(625,10,bias=True)
    torch.nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self,x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=self.layer3(out)
    out=out.view(out.size(0),-1)
    out=self.layer4(out)
    out=self.fc2(out)
    return out

In [6]:
model=CNN().to(device)


In [7]:
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)


In [9]:
data_loader=torch.utils.data.DataLoader(dataset=mnist_train,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True)

In [10]:
total_batch=len(data_loader)
print('총 배치의 수 {}'.format(total_batch))

총 배치의 수 600


In [11]:
for epoch in range(training_epochs):
  avg_cost=0

  for X, Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)

    optimizer.zero_grad()
    hypothesis=model(X)
    cost=criterion(hypothesis,Y)
    cost.backward()
    optimizer.step()

    avg_cost+=cost/total_batch

  print('[Epoch : {:>4}] cost = {:>.9}'.format(epoch+1,avg_cost))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[Epoch :    1] cost = 0.191855296
[Epoch :    2] cost = 0.0532080606
[Epoch :    3] cost = 0.036841549
[Epoch :    4] cost = 0.0294816215
[Epoch :    5] cost = 0.0244360529
[Epoch :    6] cost = 0.0198735986
[Epoch :    7] cost = 0.0176967513
[Epoch :    8] cost = 0.0143514359
[Epoch :    9] cost = 0.0127636464
[Epoch :   10] cost = 0.010939789
[Epoch :   11] cost = 0.0116941743
[Epoch :   12] cost = 0.00914803147
[Epoch :   13] cost = 0.00595878018
[Epoch :   14] cost = 0.00994120166
[Epoch :   15] cost = 0.00710024638


In [12]:
with torch.no_grad():
  X_test=mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
  Y_test=mnist_test.test_labels.to(device)


  prediction = model(X_test)
  correct_prediction=torch.argmax(prediction,1)==Y_test
  accuracy=correct_prediction.float().mean()
  print('Accuracy',accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy 0.9842000007629395
